In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# ensure the backend is set
import os
if "KERAS_BACKEND" not in os.environ:
    # set this to "torch", "tensorflow", or "jax"
    os.environ["KERAS_BACKEND"] = "jax"

import keras

# for BayesFlow devs: this ensures that the latest dev version can be found
import sys
sys.path.append('../')

import bayesflow as bf
from bayesflow.data_adapters import DataAdapter
from bayesflow.simulators import make_simulator

In [2]:
# this makes the meta simulator batched
# so we can amortized over the sample size N
def meta(batch_shape):
  n = np.random.randint(8, 12)
  return dict(n=n)

def prior():
    mu = np.random.normal(0.0, 1.0)
    sigma = np.random.gamma(1, 1)
    return dict(mu=mu, sigma=sigma)

def likelihood(mu, sigma, n):
    y = np.random.normal(mu, sigma, size=n)
    return dict(y=y)

In [3]:
# TODO: perhaps improve the argument name "meta_fn"?
simulator = bf.simulators.make_simulator([prior, likelihood], meta_fn = meta)

In [4]:
# generate a batch of two training samples
# TODO: please allow 2 instead of (2, )
sample_data = simulator.sample((2, ))
print(sample_data["mu"])

[[-0.11223909]
 [-0.05045865]]
[[-0.11223909]
 [-0.05045865]]


In [13]:
data_adapter = DataAdapter() \
    .to_array() \
    .broadcast("n") \
    .as_set("y") \
    .convert_dtype(from_dtype="float64", to_dtype="float32") \
    .standardize(exclude="n") \
    .concatenate(["mu", "sigma"], into="inference_variables") \
    .rename("n", "inference_conditions") \
    .rename("y", "summary_variables")

In [14]:
processed_data = data_adapter(sample_data, batch_size=2)
processed_data["inference_conditions"].shape

(2, 1)

In [15]:
inference_network = bf.networks.FlowMatching(
    subnet="mlp",
    subnet_kwargs=dict(
        depth=6,
        width=256,
    ),
)

In [16]:
approximator = bf.ContinuousApproximator(
   inference_network=inference_network,
   summary_network=bf.networks.DeepSet(),
   data_adapter=data_adapter,
)

# TODO: can we add the data_adapter *only* after creating approximator?

In [17]:
learning_rate = 1e-4
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
approximator.compile(optimizer=optimizer)

In [18]:
history = approximator.fit(
    epochs=10,
    num_batches = 512,
    batch_size = 64,
    simulator = simulator,
)

ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (1,8)  and requested shape (64,10)

In [ ]:
# Set the number of posterior draws you want to get
num_samples = 5000

# Obtain samples from amortized posterior
n = 10
y = np.random.normal(1, 1, size=(n, 1))
conditions = {"y": y, "n": n}
keras.tree.map_structure(keras.ops.shape, conditions)
pdraws = approximator.sample(conditions=conditions, num_samples = num_samples)
pdraws = np.append(pdraws["mu"], pdraws["sigma"], axis = -1)
pdraws

In [ ]:
# Prepare figure
f, axes = plt.subplots(1, figsize=(6, 6))

# Plot samples
axes.scatter(pdraws[:, 0], pdraws[:, 1], color="#153c7a", alpha=0.75, s=0.5)
sns.despine(ax=axes)
axes.set_title(r"Posterior draws")
axes.grid(alpha=0.3)
axes.set_aspect("equal", adjustable="box")
axes.set_xlim([0, 4])
axes.set_ylim([0, 5])